In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import plotly
import plotly.express as px

In [262]:
#loading datasets
order_list = pd.read_csv("dataset/olist_orders_dataset.csv")
order_reviews = pd.read_csv("dataset/olist_order_reviews_dataset.csv")
order_list_itens = pd.read_csv("dataset/olist_order_items_dataset.csv")
custumer_df = pd.read_csv("dataset/olist_customers_dataset.csv")
geoloc = pd.read_csv("dataset/olist_geolocation_dataset.csv")
sellers = pd.read_csv("dataset/olist_sellers_dataset.csv")
products =  pd.read_csv("dataset/olist_products_dataset.csv")

### Analisando DataSets sobre ordens e suas avaliacoes

In [3]:
order_reviews.head(10)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
5,15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,NaN,NaN,2018-04-13 00:00:00,2018-04-16 00:39:37
6,07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5,NaN,NaN,2017-07-16 00:00:00,2017-07-18 19:30:34
7,7c6400515c67679fbee952a7525281ef,c31a859e34e3adac22f376954e19b39d,5,NaN,NaN,2018-08-14 00:00:00,2018-08-14 21:36:06
8,a3f6f7f6f433de0aefbb97da197c554c,9c214ac970e84273583ab523dfafd09b,5,NaN,NaN,2017-05-17 00:00:00,2017-05-18 12:05:37
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47


In [11]:
#visualizacao da distribuicao das availiacoes 
fig = px.histogram(order_reviews, x="review_score",title='Review Distribution')
fig.show()

 Podemos observar que a grande marioria das reviews sao positivas porem observa-se mais de 10 mil reviews negativas. 

In [5]:
### analizando palavras presentes nas ordens
num_total = len(order_reviews['review_comment_message'])
num_message = len(order_reviews['review_comment_message'].dropna())
print(f"Numero total de orderns: {num_total}, Numero de orderns com Review:{num_message}")

Numero total de orderns: 99224, Numero de orderns com Review:40977


In [194]:
def Contar_palavras_em_documetos(documents,ngramas_start,ngramas_end):
    #faz uma matrix esparca das plavras presentes nos documetos 
    reviews = [s for s in documents['review_comment_message'].dropna()]
    sw = nltk.corpus.stopwords.words('portuguese')
    vectorizer = CountVectorizer(max_features=5000,binary = True,stop_words=sw,ngram_range = (ngramas_start,ngramas_end))
    # Soma o numero de vezes que a palavra apareceu em todos os documentos
    x = vectorizer.fit_transform(reviews)
    x = np.sum(x,axis=0)
    #Monta tuplas no formato (palavra,numero de x que apareceu)
    tuplas = [(x[0,vectorizer.vocabulary_[i]], i) for i in vectorizer.vocabulary_.keys()]
    tuplas_ordenadas = sorted(tuplas, reverse=True)
    palavras = [ t[1] for t in tuplas_ordenadas ]
    contagens = [ t[0] for t in tuplas_ordenadas ]
    return [palavras,contagens]
    

In [44]:
palavras,contagens = Contar_palavras_em_documetos(order_reviews,1,1)

In [45]:
#Plota grafico numero de aparicoes x palavra
fig = px.bar(x=palavras[0:30], y=contagens[0:30])
fig.update_layout(
    title="Contagem de Palavras em documetos",
    xaxis_title="Palavra",
    yaxis_title="Numero de aparicoes em documentos",
)

fig.show()

Podemos observar que as palavras mais utilizadas em reviews sao relacionadas aos produtos e a entrega, algo que ja deveria ser esperado. Para a proxima fase da analise iremos classicar reviews positivas como nota>=4 e reviews  negataivas como nota<=3.Para que possamos analisar quais palavras estao sendo associadas a reviews boas e ruins 

In [28]:
#tirando reviews sem comentarios
order_reviews.dropna(subset=["review_comment_message"],inplace = True)

In [33]:
good_reviews = order_reviews[order_reviews["review_score"]>=4]
bad_reviews = order_reviews[order_reviews["review_score"]<=3]

In [35]:
print(f"Numero total de boas reviews: {len(good_reviews)}, Numero total de reviews ruins:{len(bad_reviews)}")

Numero total de boas reviews: 26530, Numero total de reviews ruins:14447



podemos obeservar que existem mais reviews positivas do que negativas

In [231]:
palavras_b,contagens_b = Contar_palavras_em_documetos(good_reviews,1,1)
fig1 = px.bar(x=palavras_b[0:30], y=contagens_b[0:30])
fig1.update_layout(
    xaxis_title="Palavra",
    yaxis_title="Numero de aparicoes em documentos",
)
#ff6361
fig1.update_traces(marker_color='#56b4c8')
print("loaded_figure1")

loaded_figure1


In [232]:
palavras_bad,contagens_bad = Contar_palavras_em_documetos(bad_reviews,1,1)
fig2 = px.bar(x=palavras_bad[0:30], y=contagens_bad[0:30])
fig2.update_layout(
    xaxis_title="Palavra",
    yaxis_title="Numero de aparicoes em documentos"
)
fig2.update_traces(marker_color='#ff6361')
print("loaded_figure2")

loaded_figure2


In [233]:
from plotly.subplots import make_subplots
titulo_b = "Contagem de Palavras em documetos com boas reviews"
titulo_r = "Contagem de Palavras em documetos com reviews ruins"
fig = make_subplots(rows=2,vertical_spacing = 0.2,subplot_titles=(titulo_b, titulo_r))

In [234]:
fig.add_trace(fig1['data'][0], row=1,col = 1)
fig.add_trace(fig2['data'][0], row=2,col = 1)
fig.update_layout(height=900, width=800)
fig.update_yaxes(title_text="Aparicoes em documentos",row=1, col=1)
fig.update_yaxes(title_text="Aparicoes em documentos",row=2, col=1)
fig.show()

Podemos observar que apesar dos resultados apresentarem palavras diferentes, em geral os resultados sao bem similares, isso acaba impedindo que qualquer interpretacao relevante seja realizada. Por essa razao iremos observar em qual contexto essas palavras foram usadas, aplicando trigamas nos documentos

In [212]:
palavras_b,contagens_b = Contar_palavras_em_documetos(good_reviews,3,3)
fig1 = px.bar(x=palavras_b[0:30], y=contagens_b[0:30])
fig1.update_layout(
    xaxis_title="Palavra",
    yaxis_title="Numero de aparicoes em documentos",
)
#ff6361
fig1.update_traces(marker_color='#56b4c8')
print("loaded_figure1")

loaded_figure1


In [213]:
palavras_bad,contagens_bad = Contar_palavras_em_documetos(bad_reviews,3,3)
fig2 = px.bar(x=palavras_bad[0:30], y=contagens_bad[0:30])
fig2.update_layout(
    xaxis_title="Palavra",
    yaxis_title="Numero de aparicoes em documentos"
)
fig2.update_traces(marker_color='#ff6361')
print("loaded_figure2")

loaded_figure2


In [221]:
titulo_b = "Contagem de Palavras em documetos com boas reviews"
titulo_r = "Contagem de Palavras em documetos com reviews ruins"
fig = make_subplots(rows=2,vertical_spacing = 0.29,subplot_titles=(titulo_b, titulo_r))
fig.add_trace(fig1['data'][0], row=1,col = 1)
fig.add_trace(fig2['data'][0], row=2,col = 1)
fig.update_layout(height=900, width=800)
fig.update_yaxes(title_text="Aparicoes em documentos",row=1, col=1)
fig.update_yaxes(title_text="Aparicoes em documentos",row=2, col=1)
fig.show()

Podemos observar resultados muito melhores utilizando tri gramas. Isso pode ser explicado pois  tri gramas nos dao um contexto muito maior de onde as palavras estao aparecendo. Assim podemos observar que consumidores santisfeitos elogiaram a data de entrega e a qualidade do produto. Em compensacao consumidores instasfeitos criticaram atraso, qualidade ou nao recebimento do produto.

In [235]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

In [246]:
print("Porter Stemmer")
porter = PorterStemmer()
print(porter.stem("cats and birds"))
print(porter.stem("trouble"))
print(porter.stem("troubling"))
print(porter.stem("troubled"))
print(porter.stem("runs"))
print("Lancaster Stemmer")

Porter Stemmer
cats and bird
troubl
troubl
troubl
run
Lancaster Stemmer


In [265]:
#bad_reviews.merge(order_list,on="order_id",how="left")

In [260]:
all(order_list["customer_id"].duplicated())

False

In [277]:
order_list_itens

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [298]:
for index,val in enumerate(order_list_itens["order_id"].duplicated()):
    if(val ==True):
        print(index)
        break

14


In [301]:
print(order_list.iloc[14])
order_list[order_list["order_id"]=="432aaf21d85167c2c86ec9448c4e42cc"]

order_id                         432aaf21d85167c2c86ec9448c4e42cc
customer_id                      3df704f53d3f1d4818840b34ec672a9f
order_status                                            delivered
order_purchase_timestamp                      2018-03-01 14:14:28
order_approved_at                             2018-03-01 15:10:47
order_delivered_carrier_date                  2018-03-02 21:09:20
order_delivered_customer_date                 2018-03-12 23:36:26
order_estimated_delivery_date                 2018-03-21 00:00:00
Name: 14, dtype: object


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
14,432aaf21d85167c2c86ec9448c4e42cc,3df704f53d3f1d4818840b34ec672a9f,delivered,2018-03-01 14:14:28,2018-03-01 15:10:47,2018-03-02 21:09:20,2018-03-12 23:36:26,2018-03-21 00:00:00


In [307]:
order_list_itens["order_id"].is_unique  

False

In [312]:
order_list_itens[order_list_itens["order_id"].duplicated()].pivot_table(index=['order_id'], aggfunc='size')

order_id
0008288aa423d2a3f00fcb17cd7d8719    1
00143d0f86d6fbd9f9b38ab440ac16f5    2
001ab0a7578dd66cd4b0a71f5b6e1e41    2
001d8f0e34a38c37f7dba2a37d4eba8b    1
002c9def9c9b951b1bec6d50753c9891    1
                                   ..
ffd84ab39cd5e873d8dba24342e65c01    1
ffe4b41e99d39f0b837a239110260530    1
ffecd5a79a0084f6a592288c67e3c298    2
fff8287bbae429a99bb7e8c21d151c41    1
fffb9224b6fc7c43ebb0904318b10b5f    3
Length: 9803, dtype: int64

In [314]:
order_list_itens[order_list_itens["order_id"] == "0008288aa423d2a3f00fcb17cd7d8719"]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
13,0008288aa423d2a3f00fcb17cd7d8719,1,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.9,13.37
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.9,13.37
